In [0]:
import requests
import pandas as pd
from datetime import datetime
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

# 🔑 Token da SPTrans
TOKEN = "725dfd912a4526086c542731327cfd72341ad64352761c16aad6ff954fa4bbb1"

# 🔐 Autenticação
auth = requests.post(f"https://api.olhovivo.sptrans.com.br/v2.1/Login/Autenticar?token={TOKEN}")
if auth.status_code != 200:
    raise Exception("Erro na autenticação. Verifique seu token.")


In [0]:
# ⏱️ Horário da consulta
hora_consulta = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

In [0]:
# 🔁 Pegar lista de códigos da sua tabela silver.api_linhas
df_linhas = spark.sql("SELECT DISTINCT codigo_linha FROM sp_trans2.silver.api_linhas where regiao = 'Centro'")
codigos_linha = [row['codigo_linha'] for row in df_linhas.collect()]


In [0]:
# 📊 Lista pra armazenar resultados
resultados = []

for codigo in codigos_linha:
    url = f"https://api.olhovivo.sptrans.com.br/v2.1/Posicao/Linha?codigoLinha={codigo}"
    resp = requests.get(url, cookies=auth.cookies)
    
    if resp.status_code == 200:
        try:
            dados = resp.json()
            veiculos = dados.get("vs", [])
            qtd = len(veiculos)
            
            resultados.append({
                "codigo_linha": codigo,
                "qtd_veiculos": qtd,
                "hora_consulta": hora_consulta
            })
        except Exception as e:
            print(f"⚠️ Erro ao processar linha {codigo}: {e}")
    else:
        print(f"❌ Erro HTTP {resp.status_code} para linha {codigo}")

# 🧾 Criar DataFrame pandas
df = pd.DataFrame(resultados)
# display(df)

In [0]:
df_spark = spark.createDataFrame(df)
df_spark.write.mode("append").saveAsTable("sp_trans2.bronze.api_veiculos_ativos")